In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
pruning_ratio = 20 # edit
pruning_method = "magnitude" # edit, can also be "random" 

In [4]:
import os
import pandas as pd
import torch
from dataset_preprocessing import TokenInfo
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import itertools
import pandas as pd
from tqdm import tqdm

/home/research/andriv/.conda/envs/pytorch_cuda-11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Importances

## Model

In [5]:
model_id = "microsoft/phi-1_5"
model_revision = "349cf8b5e81fd5f791d1740da5de1313a0419bbd" # latest as of feb 1st

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

In [7]:
vocab = tokenizer.get_vocab()
len(vocab)

50295

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=model_revision,
    trust_remote_code=True,
    # be careful with this?
    # torch_dtype=torch.float16,
    # attn_implementation="flash_attention_2",
)

## Prune Model

In [9]:
from baseline_pruners import prune_mlps, prune_mlp_magnitude, prune_mlp_random
from importances import get_mlps

In [10]:
mlps = get_mlps(model)

In [11]:
pruning_fn = prune_mlp_random if pruning_method == 'random' else prune_mlp_magnitude

In [12]:
prune_mlps(mlps, pruning_ratio/100, pruning_fn)

In [13]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=6554, bias=True)
          (fc2): Linear(in_features=6554, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e

## Metric Callback

In [14]:
from transformers import TrainerCallback

In [15]:
from evaluation import evaluate_on_nlp_tasks

In [16]:
class AccEvalCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.last_step=-1

    def on_evaluate(self, args, state, control, model, **kwargs):
        if state.global_step == self.last_step:
            return
        self.last_step = state.global_step
        train = model.training
        model.eval()
        with torch.no_grad():
            os.environ["TQDM_DISABLE"] = "1"
            eval_res = evaluate_on_nlp_tasks(model, tokenizer, limit=100, do_shuffle=True)["results"]
            # import pdb; pdb.set_trace()
            eval_res = {k:v["acc,none"] for k,v in eval_res.items()}
            for k, v in eval_res.items():
                state.log_history.append(
                    {
                        k:v,
                        "epoch":state.epoch,
                        "step":state.global_step,
                    }
                )
            del os.environ['TQDM_DISABLE']
            print(eval_res)
        model.train(train)

In [17]:
class SaveCallback(TrainerCallback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path
        self.last_step=-1

    def on_evaluate(self, args, state, control, model, **kwargs):
        if state.global_step == self.last_step:
            return
        self.last_step = state.global_step
        try:
            torch.save(model.state_dict(), self.save_path)
        except Exception as e:
            print(f"error saving {e}")

## Train model

In [18]:
from peft import LoraConfig, PeftConfig
import transformers

In [19]:
from post_training import get_lora_config, get_training_arguments
from dataset import get_baseline_dataset
from trl import SFTTrainer

In [20]:
lora_config = get_lora_config()
training_arguments = get_training_arguments("./tmp")

In [21]:
model.cuda();

In [22]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [23]:
dataset = get_baseline_dataset()
train_data, eval_data = dataset["train"], dataset["test"]

reading pickle


In [24]:
tokenizer.pad_token = tokenizer.eos_token
training_arguments.save_strategy="no"
training_arguments.eval_steps = 100

In [25]:
save_path = f"./tmp/{pruning_method}_{pruning_ratio}_state_dict"
callbacks = [AccEvalCallback(), SaveCallback(save_path)]

In [26]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    dataset_text_field="text",
    max_seq_length=1024, # tweak this
    # TODO: think harder about the datacollator
    # data_collator=transformers.DataCollatorForSeq2Seq(
    #     tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    # ),
    callbacks=callbacks,
)

Map: 100%|██████████| 2000/2000 [00:01<00:00, 1743.68 examples/s]
2024-03-11:19:05:27,152 WARNING  [logging.py:61] Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Before training performance evaluation

In [27]:
from evaluation import evaluate_on_nlp_tasks

In [ ]:
trainer.evaluate()

wandb: Currently logged in as: andriv (andriai). Use `wandb login --relogin` to force relogin


2024-03-11:19:09:31,428 WARNING  [huggingface.py:105] `pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
2024-03-11:19:09:31,454 WARNING  [huggingface.py:315] Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration


will shuffle dataset


In [ ]:
model.eval()

In [ ]:
eval_res_orig = evaluate_on_nlp_tasks(model, tokenizer, limit=1000, bootstrap_iters=1000, do_shuffle=False)

In [ ]:
eval_res_orig["results"]

In [ ]:
eval_res = evaluate_on_nlp_tasks(model, tokenizer, limit=1000, bootstrap_iters=1000, do_shuffle=True)

In [ ]:
eval_res["results"]

## Training

In [ ]:
model.train()

In [ ]:
train_res = trainer.train()

In [ ]:
pd.to_pickle(trainer.state, f"./tmp/{pruning_method}_{pruning_ratio}.pkl")

In [ ]:
trainer_state = trainer.state
pd.DataFrame(trainer_state.log_history).tail()

## Post-training performance evaluation:

In [ ]:
valid_loss = pd.DataFrame(trainer_state.log_history)[["step", "eval_loss"]].set_index("step").dropna()

In [ ]:
valid_loss

In [ ]:
valid_loss.plot()

In [ ]:
from evaluation import evaluate_on_nlp_tasks

In [ ]:
model.eval();

In [ ]:
with torch.no_grad():
    eval_res = evaluate_on_nlp_tasks(model, tokenizer, limit=300, do_shuffle=True)

In [ ]:
eval_res["results"]

In [ ]:
eval_res_orig = evaluate_on_nlp_tasks(model, tokenizer, limit=1000, bootstrap_iters=1000, do_shuffle=False)

In [ ]:
eval_res_orig["results"]

In [ ]:
eval_res = evaluate_on_nlp_tasks(model, tokenizer, limit=1000, bootstrap_iters=1000, do_shuffle=True)

In [ ]:
eval_res["results"]

# Save

In [ ]:
model.cpu();

In [ ]:
torch.save(model.state_dict(), save_path)

# Stats

In [ ]:
import pandas as pd

In [ ]:
trainer_state = pd.read_pickle(f"tmp_logs/{pruning_method}_{pruning_ratio}.pkl")

In [ ]:
df = pd.DataFrame(trainer_state.log_history)

In [ ]:
metrics_df = df[["step", "hellaswag", "piqa", "boolq", "winogrande"]]

In [ ]:
metrics_df.ffill().tail()

In [ ]:
metrics_df[["step", "boolq"]].dropna().set_index("step").plot()

In [ ]:
metrics_df[["step", "hellaswag"]].dropna().set_index("step").plot()

In [ ]:
metrics_df = df[["step", "piqa"]].dropna().set_index("step").plot()

In [ ]:
metrics_df = df[["step", "winogrande"]].dropna().set_index("step").plot()